### Converter os dados Para Parquet para máxima performance no dashboard

In [1]:
import pandas as pd
import os
import re

# Configuração de caminhos
BASE_DIR = r"C:\Users\pedro\Downloads\python_gis\int_mercado\example6"
INPUT_FILE = os.path.join(BASE_DIR, "Study_Case_Competitors_Brazil.xlsx")
OUTPUT_FILE = os.path.join(BASE_DIR, "competitors_processed.parquet")

def classify_tier(capital):
    if capital <= 20000: return 'Micro Corretor'
    elif capital <= 100000: return 'PME (Concorrente Direto)'
    elif capital <= 1000000: return 'Assessoria/Consolidadora'
    else: return 'Big Player/Multinacional'

def classify_threat(row):
    capital = row['capital_social']
    idade = row['idade_empresa_anos']
    if capital > 1000000 and idade < 5: return 'Insurtech'
    if capital > 1000000: return 'Dominante'
    if idade > 15: return 'Tradicional'
    if idade < 3: return 'Novo Entrante'
    return 'Padrão'

print("Lendo Excel...")
df = pd.read_excel(INPUT_FILE, engine='openpyxl')

print("Aplicando Engenharia de Atributos...")
df['capital_social'] = pd.to_numeric(df['capital_social'], errors='coerce').fillna(0)
df['idade_empresa_anos'] = ((pd.Timestamp.now() - pd.to_datetime(df['data_inicio_atividade'], errors='coerce')).dt.days / 365.25).round(1)

df['tier_concorrente'] = df['capital_social'].apply(classify_tier)
df['perfil_ameaca'] = df.apply(classify_threat, axis=1)

# Limpeza de texto para filtros
df['uf_norm'] = df['uf'].astype(str).str.upper()
df['municipio_norm'] = df['municipio'].astype(str).str.title()

# Salvando em Parquet (Alta performance)
df.to_parquet(OUTPUT_FILE, index=False)
print(f"Sucesso! Salvo em: {OUTPUT_FILE}")

Lendo Excel...
Aplicando Engenharia de Atributos...
Sucesso! Salvo em: C:\Users\pedro\Downloads\python_gis\int_mercado\example6\competitors_processed.parquet
